In [1]:
1 + 1

2

In [2]:
dev_mode = false

using Distributed

if !dev_mode
    if nprocs() == 1 addprocs(4) end
end

using MLToolkit: flatten_dict, dict2namedtuple
@everywhere using Random: seed!

if dev_mode
    using Revise
end

@everywhere using RMMMDNets

;

In [11]:
dataset = "gaussian"
# dataset = "ring"
# dataset = "mnist"

@everywhere data = get_data($dataset)

# model_name = "mmdnet"
model_name = "rmmmdnet"

include("scripting.jl")

args_exp_name_list = if dev_mode
    args, exp_name = get_args(
        dataset, 
        model_name;
        override=Dict{Symbol,Any}(), suffix=""
    )
    [(args, exp_name)]
else
    let d=dataset, m=model_name
        [get_args(d, m; override=Dict(:D_h => D_h)) for D_h in ["10,10", "20,20", "100,100"]]
    end
end

;

┌ Info: On dataset gaussian with args
│   batch_size_gen = 500
│   batch_size = 500
│   σ_last = identity
│   D_z = 5
│   D_fx = 2
│   dataset = gaussian
│   σ = tanh
│   opt = adam
│   model_name = rmmmdnet
│   n_epochs = 200
│   D_h = 10,10
│   σs = 1,2
│   lr = 0.001
└ @ Main /afs/inf.ed.ac.uk/user/s16/s1672897/projects/RMMMDNets.jl/scripting.jl:73
┌ Info: On dataset gaussian with args
│   batch_size_gen = 500
│   batch_size = 500
│   σ_last = identity
│   D_z = 5
│   D_fx = 2
│   dataset = gaussian
│   σ = tanh
│   opt = adam
│   model_name = rmmmdnet
│   n_epochs = 200
│   D_h = 20,20
│   σs = 1,2
│   lr = 0.001
└ @ Main /afs/inf.ed.ac.uk/user/s16/s1672897/projects/RMMMDNets.jl/scripting.jl:73
┌ Info: On dataset gaussian with args
│   batch_size_gen = 500
│   batch_size = 500
│   σ_last = identity
│   D_z = 5
│   D_fx = 2
│   dataset = gaussian
│   σ = tanh
│   opt = adam
│   model_name = rmmmdnet
│   n_epochs = 200
│   D_h = 100,100
│   σs = 1,2
│   lr = 0.001
└ @ Main /afs/inf.e

In [ ]:
@everywhere function run_exp(args, exp_name)
    seed!(1234)

    model = get_model(args.model_name, args, data, exp_name)

    vis(data, model)

    dataloader = DataLoader(data, args.batch_size)
    
    train!(model, args.n_epochs, dataloader)
end

@sync @distributed for (args, exp_name) in args_exp_name_list
    run_exp(args, exp_name)
end

;

┌ Info: Init rmmmdnet with 1,264 parameters
└   logdir = "gaussian/rmmmdnet/opt=adam-σ=tanh-D_h=20,20-σs=1,2-D_fx=2-D_z=5/20092019-23-14-4"
┌ Info: Init rmmmdnet with 434 parameters
└   logdir = "gaussian/rmmmdnet/opt=adam-σ=tanh-D_h=10,10-σs=1,2-D_fx=2-D_z=5/20092019-23-14-4"
┌ Info: Init rmmmdnet with 22,304 parameters
└   logdir = "gaussian/rmmmdnet/opt=adam-σ=tanh-D_h=100,100-σs=1,2-D_fx=2-D_z=5/20092019-23-14-4"
